episodic_only
- mean 44.56
- std 1.85

semantic_only
- mean 55.72
- std 1.61

random
- mean 37.64 
- std 2.16

dqn, pretrained, no ddqn, no dueling (w/ warm_start)
- mean 108.2
- std 5.01

dqn, no ddqn, no dueling (w/ warm_start)
- mean 91.28
- std 8.43

The code with AAAI paper (not_pretrained, pretrained):

- DQN:            110.7, 89.3
- DDQN:           108.2, 90.2
- Dueling DQN:    102.7, 81.4
- Dueling DDQN:   109.3, 89.0

After writing tests (pre_trained)
- mean: 103.84
- std: 5.34

After writing tests (not_pre_trained)
- mean: 83.96
- std: 4.21

Alright I don't know why I can't reproduce the original results. The performance is always about 7 points lower.

In [11]:
from explicit_memory.utils import read_yaml
from glob import glob

test_scores = {"pre_trained": [], "not_pre_trained": []}
for foo in glob("./training_results/after-writing-tests/pre-trained/*/results.yaml"):
    results = read_yaml(foo)
    test_scores["pre_trained"].append(results["test_score"])

for foo in glob(
    "./training_results/after-writing-tests/not-pre-trained/*/results.yaml"
):
    results = read_yaml(foo)
    test_scores["not_pre_trained"].append(results["test_score"])

In [12]:
test_scores

{'pre_trained': [{'mean': 109.8, 'std': 5.69},
  {'mean': 96.0, 'std': 5.66},
  {'mean': 109.8, 'std': 4.94},
  {'mean': 100.8, 'std': 7.0},
  {'mean': 102.8, 'std': 7.81}],
 'not_pre_trained': [{'mean': 89.4, 'std': 7.9},
  {'mean': 83.8, 'std': 10.49},
  {'mean': 87.8, 'std': 5.9},
  {'mean': 78.2, 'std': 10.86},
  {'mean': 80.6, 'std': 5.94}]}

In [13]:
import numpy as np

np.mean([foo["mean"] for foo in test_scores["pre_trained"]]), np.std(
    [foo["mean"] for foo in test_scores["pre_trained"]]
)

(103.84, 5.3447544377641885)

In [14]:
np.mean([foo["mean"] for foo in test_scores["not_pre_trained"]]), np.std(
    [foo["mean"] for foo in test_scores["not_pre_trained"]]
)

(83.96, 4.215020759142238)

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True

from agent import DQNAgent

for pretrain_semantic in [False, True]:
    for test_seed in [0, 1, 2, 3, 4]:
        all_params = {
            "env_str": "room_env:RoomEnv-v1",
            "max_epsilon": 1.0,
            "min_epsilon": 0.1,
            "epsilon_decay_until": 128 * 16,
            "gamma": 0.65,
            "capacity": {"episodic": 16, "semantic": 16, "short": 1},
            "nn_params": {
                "hidden_size": 64,
                "num_layers": 2,
                "embedding_dim": 32,
                "v1_params": {
                    "include_human": "sum",
                    "human_embedding_on_object_location": False,
                },
                "v2_params": None,
            },
            "num_iterations": 128 * 16,
            "replay_buffer_size": 1024 * 128,
            "warm_start": 1024 * 128,
            "batch_size": 1024,
            "target_update_rate": 10,
            "pretrain_semantic": pretrain_semantic,
            "run_test": True,
            "num_samples_for_results": 10,
            "train_seed": test_seed + 5,
            "plotting_interval": 10,
            "device": "cpu",
            "test_seed": test_seed,
            "ddqn": False,
            "dueling_dqn": False,
        }

        # all_params = {
        #     "env_str": "room_env:RoomEnv-v1",
        #     "max_epsilon": 1.0,
        #     "min_epsilon": 0.1,
        #     "epsilon_decay_until": 128 * 16,
        #     "gamma": 0.65,
        #     "capacity": {"episodic": 4, "semantic": 4, "short": 1},
        #     "nn_params": {
        #         "hidden_size": 8,
        #         "num_layers": 2,
        #         "embedding_dim": 4,
        #         "v1_params": {
        #             "include_human": "sum",
        #             "human_embedding_on_object_location": False,
        #         },
        #         "v2_params": None,
        #     },
        #     "num_iterations": 128 * 16,
        #     "replay_buffer_size": 16,
        #     "warm_start": 16,
        #     "batch_size": 4,
        #     "target_update_rate": 10,
        #     "pretrain_semantic": pretrain_semantic,
        #     "run_test": True,
        #     "num_samples_for_results": 3,
        #     "train_seed": test_seed + 5,
        #     "plotting_interval": 10,
        #     "device": "cpu",
        #     "test_seed": test_seed,
        #     "ddqn": True,
        #     "dueling_dqn": True,
        # }

        agent = DQNAgent(**all_params)
        agent.train()

In [4]:
from explicit_memory.utils import write_yaml

write_yaml(all_params, "train.yaml")

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True

from tqdm.auto import tqdm

from agent import HandcraftedAgent

for policy in tqdm(["random", "episodic_only", "semantic_only"]):
    for test_seed in [0, 1, 2, 3, 4]:
        all_params = {
            "env_str": "room_env:RoomEnv-v1",
            "policy": policy,
            "num_samples_for_results": 10,
            "seed": test_seed,
        }
        if policy == "random":
            all_params["capacity"] = {"episodic": 16, "semantic": 16, "short": 1}
        elif policy == "episodic_only":
            all_params["capacity"] = {"episodic": 32, "semantic": 0, "short": 1}
        else:
            all_params["capacity"] = {"episodic": 0, "semantic": 32, "short": 1}

        all_params["seed"] = test_seed
        agent = HandcraftedAgent(**all_params)
        agent.test()

In [ ]:
from glob import glob
import yaml
import numpy as np

results = glob("./training_results/refactoring/episodic_only_agent/*/results.yaml")
scores = []
for file in results:
    with open(file, "r") as f:
        results = yaml.load(f, Loader=yaml.FullLoader)

    scores.append(results["test_score"]["mean"])
print(np.mean(scores), np.std(scores))

In [ ]:
from glob import glob
import yaml
import numpy as np

results = glob("./training_results/refactoring/semantic_only_agent/*/results.yaml")
scores = []
for file in results:
    with open(file, "r") as f:
        results = yaml.load(f, Loader=yaml.FullLoader)

    scores.append(results["test_score"]["mean"])
print(np.mean(scores), np.std(scores))

In [ ]:
from glob import glob
import yaml
import numpy as np

results = glob("./training_results/refactoring/random_agent/*/results.yaml")
scores = []
for file in results:
    with open(file, "r") as f:
        results = yaml.load(f, Loader=yaml.FullLoader)

    scores.append(results["test_score"]["mean"])
print(np.mean(scores), np.std(scores))